# Upload Cyber Graph into Clickhouse

In [1]:
from config import clickhouse_execute

DROP_TABLE = False
CREATE_TABLE = False

## Create Tables

In [2]:
if DROP_TABLE:
    clickhouse_execute('''DROP TABLE IF EXISTS cyberlinks''')
    clickhouse_execute('''DROP TABLE IF EXISTS relevance''')
if CREATE_TABLE:
    clickhouse_execute('''
        CREATE TABLE IF NOT EXISTS cyberlinks(
        id INTEGER,
        object_from String,
        object_to String,
        subject String,
        timestamp TIMESTAMP,
        height INTEGER,
        txhash String,
        karma INTEGER
            )
        ENGINE = ReplacingMergeTree()
        ORDER BY (id,object_from,object_to,subject)
        ''')
    clickhouse_execute('''
        CREATE TABLE IF NOT EXISTS relevance(
        id INTEGER,
        object String,
        height INTEGER,
        rank FLOAT
            )
        ENGINE = ReplacingMergeTree()
        ORDER BY (object,height)
        ''')

## Upload Data
```bash
tail +2 cyberlink.csv | clickhouse-client --query "
    INSERT INTO cyberlinks
    SELECT
        id,
        object_from,
        object_to,
        subject,
        timestamp,
        height,
        txhash,
        karma
        FROM input('id INTEGER, object_from String, object_to String, subject String, timestamp TIMESTAMP, height INTEGER, txhash String, karma INTEGER')
        FORMAT CSV"
```
```bash
tail +2 relevance.csv | clickhouse-client --query "
    INSERT INTO relevance
    SELECT
        id,
        object,
        height,
        rank
    FROM input('id INTEGER,object String, height INTEGER, rank FLOAT')
    FORMAT CSV"
```